In [1]:
%load_ext line_profiler

In [2]:
import requests
import pandas as pd

import re
import os

In [29]:
URL = "https://vi.wikipedia.org/w/api.php"
PARAMS_CAT = {
    "action": "query",
    "cmlimit": "100",
    "list": "categorymembers",
    "format": "json",
    
    "cmtitle": "",
    "cmtype": ""
}

PARAMS_WIKITEXT = {
    "action": "parse",
    "prop": "wikitext",
    "formatversion": 2,
    "format": "json",
    
    "page": ""
}

PARAMS_HTML = {
    "action": "parse",
    "formatversion": 2,
    "format": "json",
    
    "page": ""
}

SESSION = requests.Session()

In [30]:
def recurseCat(S, catdict, title, level=0, maxlevel=5):
        # RECURSIVE
    PARAMS_CAT['cmtitle'] = title
    PARAMS_CAT['cmtype'] = "subcat"
    R = S.get(url=URL, params=PARAMS_CAT)
    DATA = R.json()
    # print(DATA)
    PAGES = DATA['query']['categorymembers']

    index = 1
    for page in PAGES:
        childtitle = page['title']
        print("- "*level,index, childtitle)
        catdict.append({"cats": childtitle, "parent": title})
        
        if (level < maxlevel):
            recurseCat(S, catdict, childtitle, level+1, maxlevel)
        else:
            print("Max level reached")
        index += 1

In [135]:
import mwparserfromhell
from bs4 import BeautifulSoup

DEF_TEMPLATE_NAMES = "Thông tin|Infobox"

def template_infobox(tpl):
    return re.search(DEF_TEMPLATE_NAMES, tpl.name.strip_code(), re.IGNORECASE)

def title_to_infoboxes_names(title):
    PARAMS_WIKITEXT['page'] = title
    response = SESSION.get(url=URL, params=PARAMS_WIKITEXT)
    data = response.json()['parse']['wikitext']
        
    wikicode = mwparserfromhell.parse(data)
    templates = wikicode.filter_templates(matches=template_infobox)
    
    template_names = list(map(lambda x: str(x.name), templates))
    
    result_temp = ''.join(template_names)
    
    return result_temp

In [131]:
def title_to_infoboxes_wikicode(title):
    PARAMS_WIKITEXT['page'] = title
    response = SESSION.get(url=URL, params=PARAMS_WIKITEXT)
    data = response.json()['parse']['wikitext']
        
    wikicode = mwparserfromhell.parse(data)
    templates = wikicode.filter_templates(matches=template_infobox)
    
    # Find the infobox template
    if not templates:
        print("Template not found: " + title)
        return {}
    
    # Extract the key-value pairs from the infobox
    attrlist = {}
    for param in templates[0].params:
        val = str(param.value).strip()
        if bool(val):
            attrlist[str(param.name).strip()] = val # Key
    
    attrlist['title'] = title
    return attrlist

In [195]:
def title_to_infoboxes_html(title):
    PARAMS_HTML['page'] = title
    response = SESSION.get(url=URL, params=PARAMS_HTML)
    data = response.json()['parse']['text']
        
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(data, "html.parser")

    # Find the infobox template and extract its contents
    infobox = soup.find("table", {"class": "infobox"})
    if not infobox:
        print("Infobox not found: " + title)
        return {}
        
    rows = infobox.find_all("tr")

    # Convert the infobox contents into a Pandas DataFrame
    
    # err_key = []; err_val = [];
    
    attrlist = {}
    for row in rows:
        cell_key = row.find_all(["th"])
        cell_val = row.find_all(["td"])
        # has_table = row.find("table")
        if len(cell_key) == 1 and len(cell_val) == 1: # Deal with tables later
            key = cell_key[0].get_text().strip()
            value = cell_val[0].get_text().strip()
            attrlist[key] = value
#         else:
#             # Determine the maximum length of the lists
#             max_length = max(len(cell_key), len(cell_val))
#             for i in range(max_length):
#                 if i < len(cell_key): err_key.append(cell_key[i].get_text().strip())
#                 else: err_key.append(None);
                    
#                 if i < len(cell_val): err_val.append(cell_val[i].get_text().strip())
#                 else: err_val.append(None)
    
#     display(pd.DataFrame({'ERROR KEY': err_key, 'ERROR VAL': err_val}))
    attrlist['title'] = title
    return attrlist

In [134]:
url = 'https://vi.wikipedia.org/w/api.php?action=parse&prop=wikitext&format=json&page='
current = "Nhà Trần"

%lprun -f title_to_infoboxes_names title_to_infoboxes_names(current)

Timer unit: 1e-07 s

Total time: 0.895208 s
File: C:\Users\Legion\AppData\Local\Temp\ipykernel_16988\1952918041.py
Function: title_to_infoboxes_names at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def title_to_infoboxes_names(title):
    10         1         27.0     27.0      0.0      PARAMS_WIKITEXT['page'] = title
    11         1    7529023.0 7529023.0     84.1      response = SESSION.get(url=URL, params=PARAMS_WIKITEXT)
    12         1       6979.0   6979.0      0.1      data = response.json()['parse']['wikitext']
    13                                                   
    14         1    1170488.0 1170488.0     13.1      wikicode = mwparserfromhell.parse(data)
    15         1     245405.0 245405.0      2.7      templates = wikicode.filter_templates(matches=template_infobox)
    16                                               
    17         1        144.0    144.0      0.0      template_names = list(

In [133]:
%lprun -f title_to_infoboxes_wikicode title_to_infoboxes_wikicode(current)

Timer unit: 1e-07 s

Total time: 0.909026 s
File: C:\Users\Legion\AppData\Local\Temp\ipykernel_16988\2458799018.py
Function: title_to_infoboxes_wikicode at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def title_to_infoboxes_wikicode(title):
     2         1         31.0     31.0      0.0      PARAMS_WIKITEXT['page'] = title
     3         1    7554614.0 7554614.0     83.1      response = SESSION.get(url=URL, params=PARAMS_WIKITEXT)
     4         1      12335.0  12335.0      0.1      data = response.json()['parse']['wikitext']
     5                                                   
     6         1    1259982.0 1259982.0     13.9      wikicode = mwparserfromhell.parse(data)
     7         1     254170.0 254170.0      2.8      templates = wikicode.filter_templates(matches=template_infobox)
     8                                               
     9                                               # Find the infob

In [34]:
%lprun -f title_to_infoboxes_html title_to_infoboxes_html(current)

Timer unit: 1e-07 s

Total time: 2.05616 s
File: C:\Users\Legion\AppData\Local\Temp\ipykernel_16988\2312686346.py
Function: title_to_infoboxes_html at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def title_to_infoboxes_html(title):
     2         1         11.0     11.0      0.0      PARAMS_HTML['page'] = title
     3         1   13666783.0 13666783.0     66.5      response = SESSION.get(url=URL, params=PARAMS_HTML)
     4         1      44872.0  44872.0      0.2      data = response.json()['parse']['text']
     5                                                   
     6                                               # Parse the HTML content using BeautifulSoup
     7         1    6624595.0 6624595.0     32.2      soup = BeautifulSoup(data, "html.parser")
     8                                           
     9                                               # Find the infobox template and extract its contents
    

In [35]:
def getAllPages(S, SET_OF_CATS):
    data = []
    
    i = 0
    for cat in SET_OF_CATS:
          # GET ALL PAGES
        PARAMS_CAT['cmtitle'] = cat
        PARAMS_CAT['cmtype'] = "page"
        DATA = S.get(url=URL, params=PARAMS_CAT).json()
        
        PAGES = DATA['query']['categorymembers']
        for page in PAGES:
            page_title = page['title']
            
            if "Bản mẫu:" in page_title: # Skip bản mẫu
                continue;
                
            thispage = {"page": page_title, "cat": cat }
            data.append(thispage)
            print(thispage)
            
        i += 1
        print(f'Progress: {i}', end='\r')
            
    return pd.DataFrame(data)

## TESTING & DEFINITIONS END HERE

In [11]:
if not os.path.isfile('Categories.csv'):
    toptitle = "Thể loại:Triều đại Việt Nam"
    new_cat_dict = [{'cats': toptitle, 'parent': toptitle }]

    recurseCat(SESSION, new_cat_dict, toptitle, maxlevel=1)
    print("\nThat's all")
    
    df1 = pd.DataFrame(new_cat_dict)
    
    words_place = [
        ':Thời kỳ',
        'Nhà',
        'Triều đại',
    ]

    regex = re.compile('|:'.join(words_place), re.IGNORECASE)

    print(df1.shape); df1 = df1.drop_duplicates(subset=['cats'], keep='first')
    print(df1.shape); df1 = df1[df1['cats'].str.contains(regex)].reset_index(drop=True)
    print(df1.shape);
    
    df1.to_csv('Categories.csv', index=False)
else:
    df1 = pd.read_csv('Categories.csv')
    
df1

 1 Thể loại:Biểu tượng hoàng gia tại Việt Nam
 2 Thể loại:Người Việt Nam theo triều đại
-  1 Thể loại:Nhân vật thời Đinh
Max level reached
-  2 Thể loại:Nhân vật thời Hồ
Max level reached
-  3 Thể loại:Nhân vật thời Hậu Lê
Max level reached
-  4 Thể loại:Nhân vật thời Tiền Lê
Max level reached
-  5 Thể loại:Nhân vật thời Lý
Max level reached
-  6 Thể loại:Nhân vật thời Ngô
Max level reached
-  7 Thể loại:Nhân vật thời Nguyễn
Max level reached
-  8 Thể loại:Nhân vật thời Tây Sơn
Max level reached
-  9 Thể loại:Nhân vật Việt Nam thời kỳ Tiền độc lập
Max level reached
-  10 Thể loại:Nhân vật thời Trần
Max level reached
 3 Thể loại:An Dương Vương
-  1 Thể loại:Âu Lạc
Max level reached
 4 Thể loại:Nhà Đinh
-  1 Thể loại:Công tước nhà Đinh
Max level reached
-  2 Thể loại:Hoàng tộc nhà Đinh
Max level reached
-  3 Thể loại:Lịch sử Việt Nam thời Đinh
Max level reached
-  4 Thể loại:Nhân vật thời Đinh
Max level reached
-  5 Thể loại:Quan lại nhà Đinh
Max level reached
-  6 Thể loại:Vua nhà Đinh


,cats,parent
0,Thể loại:Triều đại Việt Nam,Thể loại:Triều đại Việt Nam
1,Thể loại:Nhà Đinh,Thể loại:Triều đại Việt Nam
2,Thể loại:Nhà Hậu Trần,Thể loại:Triều đại Việt Nam
3,Thể loại:Nhà Hồ,Thể loại:Triều đại Việt Nam
4,Thể loại:Nhà Lê trung hưng,Thể loại:Triều đại Việt Nam
5,Thể loại:Nhà Lê sơ,Thể loại:Triều đại Việt Nam
6,Thể loại:Nhà Tiền Lê,Thể loại:Triều đại Việt Nam
7,Thể loại:Nhà Lý,Thể loại:Triều đại Việt Nam
8,Thể loại:Nhà Tiền Lý,Thể loại:Triều đại Việt Nam
9,Thể loại:Nhà Mạc,Thể loại:Triều đại Việt Nam


In [12]:
if not os.path.isfile('Pages.csv'):
    all_pages = getAllPages(SESSION, df1.cats)
    
    print(all_pages.shape); allnodup = all_pages.drop_duplicates(subset=['page'], keep='first')
    
    words_place = [
        'Thời kỳ',
        'Nhà',
        'Triều đại',
        'cổ đại'
    ]

    regex = re.compile('|'.join(words_place))
    print(allnodup.shape); allnodup = allnodup[allnodup['page'].str.contains(regex)]
    
    allnodup.to_csv('Pages.csv', index=False)
else:
    allnodup = pd.read_csv('Pages.csv')

allnodup.sort_values('page')

,page,cat
17,Kênh Nhà Lê,Thể loại:Nhà Tiền Lê
5,Nhà Hậu Lê,Thể_loại:Triều đại Việt Nam
1,Nhà Hậu Trần,Thể_loại:Triều đại Việt Nam
6,Nhà Hồ,Thể_loại:Triều đại Việt Nam
2,Nhà Lê sơ,Thể_loại:Triều đại Việt Nam
3,Nhà Lê trung hưng,Thể_loại:Triều đại Việt Nam
7,Nhà Lý,Thể_loại:Triều đại Việt Nam
8,Nhà Mạc,Thể_loại:Triều đại Việt Nam
9,Nhà Nguyễn,Thể_loại:Triều đại Việt Nam
4,Nhà Ngô,Thể_loại:Triều đại Việt Nam


In [13]:
from tqdm import tqdm

In [136]:
DEF_TEMPLATE_NAMES = "Thông tin|Infobox|Cựu quốc gia"

In [137]:
tqdm.pandas()
allnodup.loc[:,'infoboxes'] = allnodup['page'].progress_apply(title_to_infoboxes_names)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:14<00:00,  1.36it/s]


In [138]:
allnodup.to_csv('Pages_Infoboxes.csv')

,page,cat,infoboxes
0,Nhà Đinh,Thể_loại:Triều đại Việt Nam,Infobox Former Country\n
1,Nhà Hậu Trần,Thể_loại:Triều đại Việt Nam,Infobox Former Country\n
2,Nhà Lê sơ,Thể_loại:Triều đại Việt Nam,Infobox Former Country\n
3,Nhà Lê trung hưng,Thể_loại:Triều đại Việt Nam,Infobox Former Country\n
4,Nhà Ngô,Thể_loại:Triều đại Việt Nam,Infobox former country\n
5,Nhà Hậu Lê,Thể_loại:Triều đại Việt Nam,
6,Nhà Hồ,Thể_loại:Triều đại Việt Nam,Infobox Former Country\n
7,Nhà Lý,Thể_loại:Triều đại Việt Nam,Cựu quốc gia\n
8,Nhà Mạc,Thể_loại:Triều đại Việt Nam,Infobox Former Country\n
9,Nhà Nguyễn,Thể_loại:Triều đại Việt Nam,Infobox former country\n


In [141]:
tqdm.pandas()
wikicode_list = allnodup['page'].progress_apply(title_to_infoboxes_wikicode)

 35%|█████████████████████████████                                                      | 7/20 [00:03<00:06,  1.90it/s]

Template not found: Nhà Hậu Lê


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [00:11<00:00,  1.60it/s]

Template not found: Kênh Nhà Lê


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:13<00:00,  1.50it/s]


In [142]:
df = pd.json_normalize(wikicode_list)
df = df.dropna(subset=['title'])
print(df.shape)

(18, 118)


In [143]:
# Sort column by most popular
counts = df.count().reset_index()
counts.columns = ['col', 'count']
counts = counts.sort_values(by='count',ascending=False)

df = df.reindex(counts['col'], axis=1)
df

col,common_languages,common_name,conventional_long_name,title,government_type,year_start,year_end,p1,s1,capital,...,flag_p2,flag_s1,flag_s2,s3,flag_s3,s4,flag_s4,s5,flag_s5,footnotes
0,[[Tiếng Hán|Hán ngữ trung đại]],Đinh triều,Đại Cồ Việt,Nhà Đinh,[[Quân chủ]],968,980,Nhà Ngô,Nhà Tiền Lê,[[Hoa Lư]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[[Tiếng Việt]]<br>[[Tiếng Hán]],Nhà Trần,Nhà Hậu Trần<br>茹後陳,Nhà Hậu Trần,[[Quân chủ chuyên chế]],1407,1414,Kỷ thuộc Minh,Kỷ thuộc Minh,[[Mô Độ]] (1407 - 1409)<br>[[Bình Than]] (1409...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[[Tiếng Việt|Việt ngữ trung đại]],Nhà Lê sơ,Đại Việt,Nhà Lê sơ,[[Quân chủ tuyệt đối]],[[1428]],[[1527]],Bắc thuộc lần 4,Nhà Mạc,[[Hà Nội|Đông Kinh]]<br/><small>(東京 [[1428]] -...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[[Tiếng Việt|Tiếng Việt trung đại]],Hậu Lê triều,Đại Việt,Nhà Lê trung hưng,[[Chế độ quân chủ]],1533,1789,Nhà Mạc,Nhà Tây Sơn,[[Kinh đô Vạn Lại - An Trường|Vạn Lại]] (tạm t...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[[Tiếng Việt và tiếng Hán trung cổ|Việt ngữ và...,Ngô,Tĩnh Hải quân,Nhà Ngô,Quân chủ,939,965,Bắc thuộc lần 3,Nhà Đinh,[[Cổ Loa]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,[[Tiếng Việt|Tiếng Việt trung đại]],Triều Hồ,Đại Ngu,Nhà Hồ,Quân chủ,1400,1407,Nhà Trần,Bắc thuộc lần 4,[[Thành Tây Đô|Tây Đô]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,[[Tiếng Việt|Tiếng Việt trung đại]],nhà Lý,Nhà Lý,Nhà Lý,Quân chủ,1009,1225,Nhà Tiền Lê,Nhà Trần,[[Hoa Lư]] (1009 - 1010)<br>[[Thăng Long]] (10...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,[[Tiếng Việt]],Nhà Mạc,<!-- See the talk page before editing please. -->,Nhà Mạc,Quân chủ,1527,1677,Nhà Lê sơ,Nhà Lê trung hưng,[[Thăng Long|Đông Kinh]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,[[Tiếng Việt]]{{br}}[[Văn ngôn|Văn ngôn Trung ...,Triều Nguyễn,Đại Việt <small>(1802–1804)</small><br />Việt ...,Nhà Nguyễn,[[Quân chủ chuyên chế]],1802,1945,Nhà Tây Sơn,Nam Kỳ{{!}}{{small|'''1862:'''}}<br />Nam Kỳ,[[Phú Xuân]] (nay là [[Huế]]),...,Seal of Nguyễn Lords.svg,Flag of France (1794-1815).svg,Flag of Central Vietnam (1885-1890).svg,Bắc Kỳ{{!}}Bắc Kỳ,Flag of Central Vietnam (1885-1890).svg,Việt Nam Dân chủ Cộng hòa{{!}}{{small|'''1945:...,Flag of North Vietnam 1945-1955.svg,Đông Dương thuộc Pháp,Flag of France (1794-1815).svg,NaN
10,[[Tiếng Việt|Tiếng Việt cổ]]\n\n[[Tiếng Trung|...,Vạn Xuân,Vạn Xuân,Nhà Tiền Lý,Chế độ quân chủ,544,602,Bắc thuộc lần 2,Bắc thuộc lần 3,[[Long Biên]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
df.to_csv('wikicodeparse.csv', index=False)
df.shape

(18, 118)

In [60]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# HTML

In [196]:
tqdm.pandas()
html_list = allnodup['page'].progress_apply(title_to_infoboxes_html)

 35%|█████████████████████████████                                                      | 7/20 [00:06<00:12,  1.08it/s]

Infobox not found: Nhà Hậu Lê


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [00:20<00:00,  1.08it/s]

Infobox not found: Kênh Nhà Lê


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:22<00:00,  1.13s/it]


In [197]:
df = pd.json_normalize(html_list)
df = df.dropna(subset=['title'])
print(df.shape)

(18, 170)


In [198]:
# Sort column by most popular
counts = df.count().reset_index()
counts.columns = ['col', 'count']
counts = counts.sort_values(by='count',ascending=False)

df = df.reindex(counts['col'], axis=1)
df

col,title,Thủ đô,Đơn vị tiền tệ,Ngôn ngữ thông dụng,Chính phủ,Lịch sử,Tôn giáo chính,Hoàng đế,Vị thế,Hiện nay là một phần của,...,• 1638-1677,• Mạc Thái Tổ soán ngôi nhà Lê sơ,• Nhà Lê trung hưng tái lập ở Ai Lao,"• Đông Kinh thất thủ, Mạc Mậu Hợp bị giết",• Tàn dư họ Mạc chấm dứt,• 1540,• 1802–1820 (đầu),• 1884–1885 (cuối cùng tronggiai đoạn độc lập),• 1926–1945 (cuối cùng),• Hồ Quý Ly soán ngôi Nhà Trần
0,Nhà Đinh,Hoa Lư,Tiền xu,Hán ngữ trung đại,Quân chủ,,"Phật giáo, Đạo giáo,...",,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Nhà Hậu Trần,Mô Độ (1407 - 1409)Bình Than (1409 - 1414),Tiền xu,Tiếng ViệtTiếng Hán,Quân chủ chuyên chế,,Tam giáo quy nguyên,NaN,Đế quốc,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nhà Lê sơ,Đông Kinh(東京 1428 - 1527),Tiền xu,Việt ngữ trung đại,Quân chủ tuyệt đối,,"Phật giáo, Nho giáo, Tín ngưỡng dân gian",,Đế quốc,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Nhà Lê trung hưng,Vạn Lại (tạm thời)(1533–1597)Đông Kinh(1597–1789),Tiền xu,Tiếng Việt trung đại,Chế độ quân chủ,,Nho giáo Phật giáo Đạo giáo Công giáo Tín ngưỡ...,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nhà Ngô,Cổ Loa,Tiền xu,Việt ngữ và Hán ngữ trung đại,Quân chủ,,"Phật giáo, Đạo giáo, Cảnh giáo[1]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Nhà Hồ,"Tây Đô20°4′B 105°36′Đ﻿ / ﻿20,067°B 105,6°Đ﻿ / ...","Thông bảo hội sao (tiền giấy), Thánh Nguyên th...",Tiếng Việt trung đại,Quân chủ,,"Phật giáo, Đạo giáo, Tín ngưỡng Việt Nam",,Đế quốc,Việt Nam Trung Quốc Lào,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Nhà Lý,Hoa Lư (1009 - 1010)Thăng Long (1010 - 1225),Tiền xu,Tiếng Việt trung đại,Quân chủ,,Phật giáo,,NaN,Việt Nam Trung Quốc Lào,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Nhà Mạc,Đông Kinh,Tiền xu,Tiếng Việt,Quân chủ,,Tam giáo quy nguyên,,Đế quốc,NaN,...,Mạc Kính Vũ,1527,1533,1592,1677,2.000.000,NaN,NaN,NaN,NaN
9,Nhà Nguyễn,"Phú Xuân (nay là Huế)16°28′B 107°36′Đ﻿ / ﻿16,4...",Quan,Tiếng ViệtVăn ngôn Trung Quốc[1][2][Ghi chú 1]...,Quân chủ chuyên chế,,"Nho giáo, Phật giáo, Công giáo, Tín ngưỡng dân...",,Đế quốc nội địa chịu sự triều cống cho Trung Q...,Campuchia Lào Trung Quốc Việt Nam,...,NaN,NaN,NaN,NaN,NaN,NaN,Gia Long,Hàm Nghi,Bảo Đại,NaN
10,Nhà Tiền Lý,Long Biên,Văn,Tiếng Việt cổ\nTiếng Hán trung đại,Chế độ quân chủ,,NaN,NaN,NaN,Việt Nam Trung Quốc Lào,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [199]:
df.to_csv('htmlparse.csv', index=False)
df.shape

(18, 170)

In [60]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)